Load API Keys

In [ ]:
from dotenv import load_dotenv
load_dotenv()

Load Contents from HLB Fixed Deposite Pages

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

Split the Loaded Data into Chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_documents(data)
texts

Initialize Pinecone

In [ ]:
import os
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)
index_name = "main-index"
namespace = "hlb-fixed-deposit-with-conversation-chain-gpt-3.5"
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

Create Embeddings and Vectors to Insert to Pinecone Database

In [ ]:
from langchain_community.vectorstores.pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = Pinecone.from_documents(texts, embeddings, index_name=index_name, namespace=namespace)

Create LLM

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

Creating Prompt Template

In [134]:
from langchain.prompts import PromptTemplate

prompt_template = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: {question}
  By only basing on this context: {context}
  If the question does not specify the fixed deposit type, answer the question assuming it is about standard fixed deposit type.
  If you don't have enought information from the document, don't answer with your general knowledge, just say you don't have that information.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, double check your answer, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the question, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the input question you to do something creative such as writing a song or making a joke.
"""
prompt = PromptTemplate.from_template(prompt_template)

Create Retriever for Vector Search

In [135]:
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

Create BM25 Retriever

In [ ]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(texts, k=5)

Create MultiQuery retriever

In [136]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
  llm=llm,
  retriever=retriever,
  )

Logging Multi Query Retriever

In [137]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

Create Ensemble Retriever

In [138]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
  retrievers=[bm25_retriever, multi_query_retriever], weights=[0.5, 0.5]
)

Compressor Pipeline

In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline, LLMChainExtractor

llm_chain_extractor = LLMChainExtractor.from_llm(llm=llm)

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0, separators=". ")
embedding_filter = EmbeddingsFilter(embeddings=embeddings)
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
pipeline_compressor = DocumentCompressorPipeline(
  transformers=[splitter, redundant_filter, embedding_filter]
)

Contextual Compression

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever

compression_retriever = ContextualCompressionRetriever(
    base_compressor=embedding_filter, base_retriever=ensemble_retriever
)

Creating Memory

In [139]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(llm=llm, memory_key="chat_history", output_key="answer", k=2, return_messages=True)

Creating Chain

In [140]:
from langchain.chains import ConversationalRetrievalChain
from langchain_community.callbacks.manager import get_openai_callback
 
chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  chain_type="stuff",
  retriever=multi_query_retriever,
  memory=memory,
  combine_docs_chain_kwargs={"prompt": prompt},
  verbose=True,
  return_source_documents=True,
  max_tokens_limit=3000
)

def ask_question(question):
  with get_openai_callback() as cb:
    result = chain.invoke(question)
    for source_documents in result["source_documents"]:
      print(f'\n{source_documents}')
    print(f'\nDocument: {len(result["source_documents"])}')
    print(f'\nAnswer: {result["answer"]}')
    print(f'{cb}\n')

Running the Chain

In [ ]:
ask_question("What is fixed deposit?")

In [141]:
ask_question("How many types of fixed deposit does HongLeong Bank provide?")

INFO:langchain.retrievers.multi_query:Generated queries: ['What are the different types of fixed deposit offered by HongLeong Bank?', 'What options for fixed deposit are available at HongLeong Bank?', 'What is the variety of fixed deposit types provided by HongLeong Bank?']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: How many types of fixed deposit does HongLeong Bank provide?
  By only basing on this context: Fixed Deposit | Hong Leong Bank Malaysia










Fixed Deposit
 
null























PERSONAL


BUSINESS



                            Priority


Priority
Islamic
Global Markets
Blue Header
Interim Homepage
Priority Banking Form
test
test
Loan Cal
Log Viewer
asdsadsad
stand alone page
Master Form Page
Master Form Page
















singapore



hong kong



vietnam



cambodia





                            ENG


LAN A
LAN B
LAN C




                            Online Banking
                        

Personal
Business













                                About Us
                            

Our Story
News & Updates
Investor Relation

In [ ]:
ask_question("What is the difference between Fixed Deposit and eFixed Deposit?")

In [142]:
ask_question("What are the interest rates for Fixed Deposit?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How many types of fixed deposit does HongLeong Bank provide?
Assistant: Hong Leong Bank provides several types of fixed deposit accounts, including:

1. Fixed Deposit
2. e-Fixed Deposit
3. Flexi Fixed Deposit
4. Senior Savers Flexi Fixed Deposit
5. Junior Fixed Deposit
6. Foreign Currency Fixed Deposit

Each type of fixed deposit account offers different features and benefits to cater to the specific needs of customers. If you have any specific questions about the interest rates, tenure, or other details of these fixed deposit accounts, please let me know and I can provide more information.
Follow Up Input: What are the interest rates for Fixed Deposit?
Standalone question:

> Finished chain.


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the current Fixed Deposit interest rates offered by Hong Leong Bank?', "2. I'm interested in learning about the specific interest rates for Fixed Deposits at Hong Leong Bank. Can you help me with that?", '3. What is the latest information on the Fixed Deposit interest rates at Hong Leong Bank?']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: What are the interest rates for Fixed Deposit at Hong Leong Bank?
  By only basing on this context: Fixed Deposit | Hong Leong Bank Malaysia










Fixed Deposit
 
null























PERSONAL


BUSINESS



                            Priority


Priority
Islamic
Global Markets
Blue Header
Interim Homepage
Priority Banking Form
test
test
Loan Cal
Log Viewer
asdsadsad
stand alone page
Master Form Page
Master Form Page
















singapore



hong kong



vietnam



cambodia





                            ENG


LAN A
LAN B
LAN C




                            Online Banking
                        

Personal
Business













                                About Us
                            

Our Story
News & Updates
Investor Rel

In [143]:
ask_question("Let's say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How many types of fixed deposit does HongLeong Bank provide?
Assistant: Hong Leong Bank provides several types of fixed deposit accounts, including:

1. Fixed Deposit
2. e-Fixed Deposit
3. Flexi Fixed Deposit
4. Senior Savers Flexi Fixed Deposit
5. Junior Fixed Deposit
6. Foreign Currency Fixed Deposit

Each type of fixed deposit account offers different features and benefits to cater to the specific needs of customers. If you have any specific questions about the interest rates, tenure, or other details of these fixed deposit accounts, please let me know and I can provide more information.
Human: What are the interest rates for Fixed Deposit?
Assistant: The interest rates for Fixed Deposit at Hong Leong Bank are as follows:

- 1 month: 2.15% p.a.
- 2 - 3 month

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How much money will I have at the end of a 12-month term if I invest RM 50,000 in Fixed Deposit?', '2. What is the final sum I can withdraw after 12 months from a Fixed Deposit investment of RM 50,000?', '3. What will be the total value available for withdrawal after a 12-month Fixed Deposit investment of RM 50,000?']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: What would be the total amount that I can withdraw at the end of a 12-month term if I invest RM 50,000 in Fixed Deposit?
  By only basing on this context: Flexibility of early fixed deposit partial withdrawal without losing interest on remaining balance

Partial withdrawal is in multiples of RM1,000. Outstanding balances will be shown on the monthly e-statement.
Effective 1 January 2019, no interest shall be payable on partially withdrawn amounts and premature withdrawals of FD.
 



                            More details

                            Less details












Convenient Withdrawal

You can withdraw from any branch nationwide. 



                            More details

                            Less details
















Eligibi